In [1]:
from model.modelorchastrator import ModelOrchestrator
import pandas as pd
from data_pipeline.dataset import Dataset

from hydra.utils import instantiate
from hydra import compose, initialize
from hydra.core.global_hydra import GlobalHydra
from data_pipeline.pipelinesteps import data_splitter

(CVXPY) Mar 27 10:27:40 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Mar 27 10:27:40 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


In [2]:
df = pd.read_feather("../../data/parquet_files/train/processed_train.feather")
df.head()

,case_id,target,date_decision,childnum_185L_mean,housetype_905L_last,familystate_447L_last,education_927M_last,cardtype_51L_last,credtype_322L_last,numberofcontrsvalue_358L_max,...,residualamount_488A_max,collater_valueofguarantee_876L_mean,pmts_overdue_1140A_max,pmts_overdue_1140A_min,pmts_overdue_1140A_mean,credacc_cards_status_52L_mode,credacc_cards_status_52L_last,last180dayaveragebalance_704A_last,last180dayaveragebalance_704A_mean,last180dayaveragebalance_704A_max
0,0,0,2019-01-03,NaN,None,None,a55475b1,None,CAL,NaN,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
1,1,0,2019-01-03,NaN,None,None,a55475b1,None,CAL,NaN,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
2,2,0,2019-01-04,NaN,None,None,a55475b1,None,CAL,NaN,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
3,3,0,2019-01-03,NaN,None,None,a55475b1,None,CAL,NaN,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN
4,4,1,2019-01-04,NaN,None,None,a55475b1,None,CAL,NaN,...,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN


In [4]:

GlobalHydra.instance().clear()
initialize(config_path="../../conf", version_base=None)
cfg = compose(config_name="config")

data_pipeline = instantiate(cfg.data_pipeline)
df = pd.read_feather("../../data/parquet_files/train/processed_train.feather")
df = data_pipeline.apply(df)

dataset = Dataset(data=df, data_splitter=data_splitter, target_column='target')

c:\Users\bartw\anaconda3\envs\credit_score\Lib\site-packages\hydra\_internal\defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [8]:
cfg = omegaconf.OmegaConf.to_yaml(cfg)

{'model_steps': [{'_target_': 'model.feature_selector.FeatureSelector', 'features': 'defaults.features.all_features'}, {'_target_': 'model.modelorchastrator.OptBinningTransformer', 'column': 'defaults.features.cat_features'}, {'_target_': 'model.modelorchastrator.HistBooster', 'learning_rate': 0.1, 'categorical_features': 'defaults.features.cat_features', 'verbose': 0, 'class_weight': 'balanced', 'max_iter': 1000, 'l2_regularization': 0.3, 'early_stopping': True, 'max_leaf_nodes': 100}]}

In [ ]:
model_orchestrator = ModelOrchestrator(cfg.model)

ConfigAttributeError: Key 'model' is not in struct
    full_key: model.model
    object_type=dict

In [ ]:
pipeline = model_orchestrator.modelpipeline

In [ ]:
dataset.X_train.head()

In [ ]:


# Split the data into training and test sets
X_train, X_test, y_train, y_test = dataset.X_train, dataset.X_test, dataset.y_train, dataset.y_test 
pipeline.fit(dataset.X_train, dataset.y_train)  

In [ ]:
from sklearn.metrics import roc_auc_score

# Predict the probabilities of the positive class
y_pred_proba = pipeline.predict_proba(X_test).T[1]

# Compute the AUC score
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"AUC score: {auc_score}")

In [ ]:
pd.Series(y_pred_proba).hist(bins=50, figsize=(10, 6))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)  
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='AUC = {:.2f}'.format(auc_score))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()


In [ ]:
test_data = pd.read_feather("../../data/parquet_files/test/processed_test.feather")
test_data.head()


In [ ]:

test_data = pd.read_feather("../../data/parquet_files/test/processed_test.feather")
test_data = data_pipeline.apply(test_data)
test_data['target'] = 0
test_dataset = Dataset(data=test_data, data_splitter=data_splitter, target_column='target')


In [ ]:
pipeline.predict_proba(test_dataset.X.reset_index())